In [1]:
## reset specific variables (replace regular_expression by the variables of interest)
#%reset_selective <regular_expression>

# reset all variables
%reset -f

In [2]:
## Importing libraries

from datetime import datetime, date, timedelta
from array import *
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from pylab import savefig
import seaborn as sns
import pandas as pd
import csv
import json
import datetime as dt
from pymongo import MongoClient
from mongoengine import *

%matplotlib inline

In [3]:
## Creating/Connecting Mongo DB instances

# Provide the mongodb atlas url to connect python to mongodb using pymongo
#CONNECTION_STRING = "mongodb+srv://<jgu>:<123>@<cluster-jgu>.mongodb.net/SMARTAttributesFilter"

connect(db='SMARTAttributesFilter', alias='SMARTAttributesFilter_alias')

connect(db='FailuresAppsLocation', alias='FailuresAppsLocation_alias')

connect(db='SMARTAtt_FailuresAppsLocation', alias='SMARTAtt_FailuresAppsLocation_alias')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [4]:
## Setting document schema

class SMARTAtt(Document):
     disk_id = FloatField(required=False, default='0')
     timestamp = DateTimeField(required=False, default='0')
     model_x = StringField(required=False, default='0')
     r_sectors = FloatField(required=False, default='0')
     u_errors = FloatField(required=False, default='0')
     p_failedA = FloatField(required=False, default='0')
     p_failedB = FloatField(required=False, default='0')
     e_failedA = FloatField(required=False, default='0')
     e_failedB = FloatField(required=False, default='0')
     n_b_written = FloatField(required=False, default='0')
     n_b_read = FloatField(required=False, default='0')
     meta = {'db_alias': 'SMARTAttributesFilter_alias'}

class FailuresAppsLocation(Document):
     disk_id = FloatField(required=False, default='0')
     failure_time = DateTimeField(required=False, default='0')
     model_x = StringField(required=False, default='0')
     model_y = StringField(required=False, default='0')
     app = StringField(required=False, default='0')
     node_id = FloatField(required=False, default='0')
     rack_id = FloatField(required=False, default='0')
     machine_room_id = FloatField(required=False, default='0')
     meta = {'db_alias': 'FailuresAppsLocation_alias'}

class SMARTAtt_FailuresAppsLocation(Document):
     smart_att = ReferenceField(SMARTAtt)
     failures_app_location = ReferenceField(FailuresAppsLocation)
     meta = {'db_alias': 'SMARTAtt_FailuresAppsLocation_alias'}

In [ ]:
# Closing the connection to the DB

disconnect(alias='SMARTAttributesFilter_alias')

disconnect(alias='FailuresAppsLocation_alias_alias')

disconnect(alias='SMARTAtt_FailuresAppsLocation_alias')

In [316]:
## Some queries in Mongoengine

## Creating the object related to the whole collection
#failuresAppsLocationTeste = FailuresAppsLocation.objects()

## Collecting the applications
apps = FailuresAppsLocation.objects().distinct("app")
print(apps)

## Looping inside dcuments, which are filtered for the kind o app
#for app_c in apps:
#   dataPerApp = FailuresAppsLocation.objects(app=app_c).filter()
   #dataPerAppToList = list(dataPerApp)
   #dataPerAppDf = pd.DataFrame.from_dict(dataPerAppToList)
     
#   for doc in dataPerApp:
#      arrayForDataPerApp = np.append(arrayForDataPerApp, doc.to_json())

##The collection size
#len(failuresAppsLocationTeste)


## Query if you know something about the document
#testando = FailuresAppsLocation.objects(disk_id="33722").get()


## Query if you know something about the document
#SMARTAttributesTest = SMARTAtt.objects(timestamp="2019-12-31").filter().limit(20)
#SMARTAttributesTest = SMARTAtt.objects(timestamp="2019-12-31").filter()

## Printing
#for i in SMARTAttributesTest:
#    print(i.timestamp)
    
## Deleting all collection
#failuresAppsLocationTeste.delete() 
  
##Deleting all collection
#for i in failuresAppsLocationTeste:
 # i.delete()

['DAE', 'DB', 'NAS', 'RM', 'SS', 'WPS', 'WS', 'WSM', 'none']


In [ ]:
## Some queries pandas

# Find an specific line based on a specific value from a column
#df_failuresLocationDocumentsJsonMTTF.loc[df_failuresLocationDocumentsJsonMTTF["mtff_node"] == 2] 

In [120]:
## Space for small tests

failuresLocationDocuments = FailuresAppsLocation.objects()

jSon_failuresLocationDocumentsJson = json.loads(failuresLocationDocuments.to_json())
df_failuresLocationDocumentsJson = pd.DataFrame.from_dict(jSon_failuresLocationDocumentsJson) 


dicDateToString = json.dumps(list(df_failuresLocationDocumentsJson['failure_time']))
dicStringToJson = json.loads(dicDateToString)
dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
df_failuresLocationDocumentsJson['failure_time'] = dicJsonToDf['$date']



#print(df_failuresLocationDocumentsJson.model_y.unique())



In [121]:
## Calculating MTTF per node, rack and general

df_failuresLocationDocumentsJsonMTTF = df_failuresLocationDocumentsJson

## Value representing: 2018-01-01 00:00:00 (starting date from the experiment)
#1514764800000

#Subctracting by the initial time of the experiment and turning into hours
df_failuresLocationDocumentsJsonMTTF['failure_time'] = df_failuresLocationDocumentsJson.failure_time.sub(1514764800000)
df_failuresLocationDocumentsJsonMTTF['failure_time'] = df_failuresLocationDocumentsJson.failure_time.div(1000)
df_failuresLocationDocumentsJsonMTTF['failure_time'] = df_failuresLocationDocumentsJson.failure_time.div(60)
df_failuresLocationDocumentsJsonMTTF['failure_time'] = df_failuresLocationDocumentsJson.failure_time.div(60)
#df_failuresLocationDocumentsJsonMTTF.head(3)

# Number of unique values by column
#id_apps = df_failuresLocationDocumentsJsonMTTF.app.unique()
#id_nodes = df_failuresLocationDocumentsJsonMTTF.node_id.unique()
id_racks = nodes = df_failuresLocationDocumentsJsonMTTF.rack_id.unique()
#id_ssds = nodes = df_failuresLocationDocumentsJsonMTTF.disk_id.unique()
print(id_racks.shape)

#Calculating the MTTFs
df_general = df_failuresLocationDocumentsJsonMTTF['failure_time'].mean()
df_appNode = df_failuresLocationDocumentsJsonMTTF.groupby(['app', 'node_id'])['failure_time'].mean()
df_appNode = pd.DataFrame(df_appNode)
df_appNode.rename(columns = {'failure_time':'mttf_appNode'}, inplace=True)
df_appRack = df_failuresLocationDocumentsJsonMTTF.groupby(['app', 'rack_id'])['failure_time'].mean()
df_appRack = pd.DataFrame(df_appRack)
df_appRack.rename(columns = {'failure_time':'mttf_appRack'}, inplace=True)
df_app = df_failuresLocationDocumentsJsonMTTF.groupby(['app'])['failure_time'].mean()
df_app = pd.DataFrame(df_app)
df_app.rename(columns = {'failure_time':'mttf_app'}, inplace=True)
df_all = df_failuresLocationDocumentsJsonMTTF.groupby(['app', 'rack_id', 'node_id'])['failure_time'].mean()

# Merging the results with the original dataframe
df_result = pd.merge(df_failuresLocationDocumentsJsonMTTF, df_appNode, how='left', on=['app', 'node_id'])
df_result = pd.merge(df_result, df_appRack, how='left', on=['app', 'rack_id'])
df_result = pd.merge(df_result, df_app, how='left', on=['app'])

# Choosing the columns of interest
df_resultFiltered = df_result.loc[:,['app','node_id', 'rack_id','mttf_appNode','mttf_appRack','mttf_app']]
df_resultFiltered.to_csv('failuresLocationApp.csv')
#df_result.head(30)



#df.groupby('Sex').sum().plot(kind='bar');
#print(df_app)
#df_appNode.groupby(['app']).plot(kind='bar')
#df.groupby(['Sex', 'Survived'] )['Survived'].count().plot.bar(figsize=(8, 6));

#d1 = data[data["City"] == "Houston"]
#dataFrame.Reg_Price[i]
#df.loc[df['column_name'] == some_value]



(4072,)


In [24]:
# Investigating Flash technology

flashtech = pd.DataFrame({
'model_y':['A3', 'A2', 'B3' , 'B2', 'C1', 'C2'],  
'flash': ['MLC', 'MLC', 'MLC' , 'MLC', '3D-TLC', '3D-TLC']})

df_flashtech = pd.merge(df_result, flashtech, how='left', on=['model_y'])

df_flashtech = df_flashtech.groupby(['app','flash'])['failure_time'].mean()

df_flashtech = pd.DataFrame(df_flashtech)

df_flashtech.rename(columns = {'failure_time':'mttf_flash'}, inplace=True)

df_flashtech.to_csv('flashtech.csv')
#df_flashtech.head()


In [28]:
# Investigating Capacity

capacity = pd.DataFrame({
'model_y':['A3', 'A2', 'B3' , 'B2', 'C1', 'C2'],  
'capacity': ['480GB', '800GB', '1920GB' , '1920GB', '1920GB', '960GB']})

df_capacity = pd.merge(df_result, capacity, how='left', on=['model_y'])

df_capacity = df_capacity.groupby(['app','capacity'])['failure_time'].mean()

df_capacity = pd.DataFrame(df_capacity)

df_capacity.rename(columns = {'failure_time':'mttf_capacity'}, inplace=True)

df_capacity.to_csv('capacity.csv')
#df_capacity.head(50)

In [ ]:
##  Creating Graphs

fig = sns.catplot(x="app", y="failure_time", hue="model_x", data=df_failuresLocationDocumentsJson,
               row="node_id", col="rack_id", kind="bar", ci=90, palette="Blues_d", aspect=0.9, height=4.5
                  , legend_out = True, margin_titles = True)

fig.set_axis_labels("App", "MTTF")
fig.set_xticklabels(["DAE", "DB", "NAS", "RM", "SS", "WPS", "WS", "WSM", "none"])

plt.savefig('teste.pdf', dpi=1200)

In [59]:
## Investigating smart attributes

## Creating the object related to the whole collection
#failuresAppsLocationTeste = FailuresAppsLocation.objects()

testando = FailuresAppsLocation.objects(disk_id="19015").get()
len(testando)

#for i in testando:
 #  print(testando)
 
 

#stocks_df[(stocks_df.Sector == "Technology") & (stocks_df.Price <500)]

#teste = df_failuresLocationDocumentsJsonMTTF[(df_failuresLocationDocumentsJsonMTTF.app == "none")]
#teste = pd.DataFrame(teste)
#teste[['failure_time']].max()
#teste = teste.loc[(teste['failure_time']) >= 17512]
#print(teste)

## Collecting the applications
#apps = FailuresAppsLocation.objects().distinct("app")
#print(apps)

## Looping inside dcuments, which are filtered for the kind o app
#for app_c in apps:
#   dataPerApp = FailuresAppsLocation.objects(app=app_c).filter()
   #dataPerAppToList = list(dataPerApp)
   #dataPerAppDf = pd.DataFrame.from_dict(dataPerAppToList)
     
#   for doc in dataPerApp:
#      arrayForDataPerApp = np.append(arrayForDataPerApp, doc.to_json())

##The collection size
#len(failuresAppsLocationTeste)


## Query if you know something about the document
#testando = FailuresAppsLocation.objects(disk_id="33722").get()


## Query if you know something about the document
#SMARTAttributesTest = SMARTAtt.objects(timestamp="2019-12-31").filter().limit(20)


9

In [63]:
#smartDocuments = SMARTAtt.objects()
smartDocuments = SMARTAtt.objects(disk_id="19015").filter()

jSon_smartDocumentsJson = json.loads(smartDocuments.to_json())
df_smartDocumentsJson = pd.DataFrame.from_dict(jSon_smartDocumentsJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee065a8ca9d5f6a1bd6c3'},19015.0,{'$date': 1514764800000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee06fa8ca9d5f6a1c0b62'},19015.0,{'$date': 1514764800000},MA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '625ee0b5a8ca9d5f6a1d6d4a'},19015.0,{'$date': 1514851200000},MA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '625ee104a8ca9d5f6a1f14fc'},19015.0,{'$date': 1514851200000},MC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '625ee10fa8ca9d5f6a1f50d3'},19015.0,{'$date': 1514851200000},MB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
#smartDocuments = SMARTAtt.objects()
smartDocumentsDB = SMARTAtt.objects(disk_id="95341").filter()

jSon_smartDocumentsDBJson = json.loads(smartDocumentsDB.to_json())
df_smartDocumentsDBJson = pd.DataFrame.from_dict(jSon_smartDocumentsDBJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsDBJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee065a8ca9d5f6a1bd6c3'},19015.0,{'$date': 1514764800000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee06fa8ca9d5f6a1c0b62'},19015.0,{'$date': 1514764800000},MA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '625ee0b5a8ca9d5f6a1d6d4a'},19015.0,{'$date': 1514851200000},MA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '625ee104a8ca9d5f6a1f14fc'},19015.0,{'$date': 1514851200000},MC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '625ee10fa8ca9d5f6a1f50d3'},19015.0,{'$date': 1514851200000},MB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#smartDocuments = SMARTAtt.objects()
smartDocumentsNAS = SMARTAtt.objects(disk_id="18144").filter()

jSon_smartDocumentsNASJson = json.loads(smartDocumentsNAS.to_json())
df_smartDocumentsNASJson = pd.DataFrame.from_dict(jSon_smartDocumentsNASJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsNASJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee041a8ca9d5f6a1b1d77'},18144.0,{'$date': 1514764800000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee0a1a8ca9d5f6a1d114d'},18144.0,{'$date': 1514764800000},MA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '625ee0b4a8ca9d5f6a1d6bd6'},18144.0,{'$date': 1514851200000},MB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '625ee125a8ca9d5f6a1fc9ae'},18144.0,{'$date': 1514851200000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '625ee166a8ca9d5f6a2134dc'},18144.0,{'$date': 1514851200000},MB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
#smartDocuments = SMARTAtt.objects()
smartDocumentsRM = SMARTAtt.objects(disk_id="99516").filter()

jSon_smartDocumentsRMJson = json.loads(smartDocumentsRM.to_json())
df_smartDocumentsRMJson = pd.DataFrame.from_dict(jSon_smartDocumentsRMJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsRMJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee04aa8ca9d5f6a1b4c15'},99516.0,{'$date': 1514764800000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee142a8ca9d5f6a206bbc'},99516.0,{'$date': 1514851200000},MC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '625ee178a8ca9d5f6a219c06'},99516.0,{'$date': 1514851200000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '625ee228a8ca9d5f6a256b3d'},99516.0,{'$date': 1514937600000},MA2,100.0,100.0,100.0,NaN,100.0,NaN,100.0,100.0
4,{'$oid': '625ee232a8ca9d5f6a25a792'},99516.0,{'$date': 1514937600000},MC1,100.0,100.0,100.0,NaN,100.0,NaN,NaN,NaN


In [68]:
#smartDocuments = SMARTAtt.objects()
smartDocumentsSS = SMARTAtt.objects(disk_id="199807").filter()

jSon_smartDocumentsSSJson = json.loads(smartDocumentsSS.to_json())
df_smartDocumentsSSJson = pd.DataFrame.from_dict(jSon_smartDocumentsSSJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsSSJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee192a8ca9d5f6a222b4f'},199807.0,{'$date': 1514851200000},MC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee22ca8ca9d5f6a258151'},199807.0,{'$date': 1514937600000},MC1,100.0,100.0,100.0,NaN,100.0,NaN,NaN,NaN
2,{'$oid': '625ee2a6a8ca9d5f6a281f52'},199807.0,{'$date': 1515024000000},MC1,100.0,100.0,100.0,NaN,100.0,NaN,NaN,NaN
3,{'$oid': '625ee40ea8ca9d5f6a2ffc3a'},199807.0,{'$date': 1515110400000},MC1,100.0,100.0,100.0,NaN,100.0,NaN,NaN,NaN
4,{'$oid': '625ee50ba8ca9d5f6a3575ff'},199807.0,{'$date': 1515196800000},MC1,100.0,100.0,100.0,NaN,100.0,NaN,NaN,NaN


In [69]:
#smartDocuments = SMARTAtt.objects()
smartDocumentsWPS = SMARTAtt.objects(disk_id="30010").filter()

jSon_smartDocumentsWPSJson = json.loads(smartDocumentsWPS.to_json())
df_smartDocumentsWPSJson = pd.DataFrame.from_dict(jSon_smartDocumentsWPSJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsWPSJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee070a8ca9d5f6a1c0f53'},30010.0,{'$date': 1514764800000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee13da8ca9d5f6a204cf9'},30010.0,{'$date': 1514851200000},MB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '625ee13da8ca9d5f6a204cfa'},30010.0,{'$date': 1514851200000},MB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '625ee173a8ca9d5f6a217d12'},30010.0,{'$date': 1514851200000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '625ee1f7a8ca9d5f6a245867'},30010.0,{'$date': 1514937600000},MB2,100.0,100.0,NaN,100.0,NaN,100.0,99.0,99.0


In [70]:
#smartDocuments = SMARTAtt.objects()
smartDocumentsWS = SMARTAtt.objects(disk_id="17304").filter()

jSon_smartDocumentsWSJson = json.loads(smartDocumentsWS.to_json())
df_smartDocumentsWSJson = pd.DataFrame.from_dict(jSon_smartDocumentsWSJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsWSJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee083a8ca9d5f6a1c73ef'},17304.0,{'$date': 1514764800000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee0a1a8ca9d5f6a1d10fe'},17304.0,{'$date': 1514764800000},MA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '625ee0b4a8ca9d5f6a1d6a5a'},17304.0,{'$date': 1514851200000},MB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '625ee12fa8ca9d5f6a200476'},17304.0,{'$date': 1514851200000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '625ee13aa8ca9d5f6a20400a'},17304.0,{'$date': 1514851200000},MB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
#smartDocuments = SMARTAtt.objects()
smartDocumentsWSM = SMARTAtt.objects(disk_id="29755").filter()

jSon_smartDocumentsWSMJson = json.loads(smartDocumentsWSM.to_json())
df_smartDocumentsWSMJson = pd.DataFrame.from_dict(jSon_smartDocumentsWSMJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsWSMJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee05ba8ca9d5f6a1ba5c1'},29755.0,{'$date': 1514764800000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee0aca8ca9d5f6a1d49e1'},29755.0,{'$date': 1514764800000},MA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '625ee0e4a8ca9d5f6a1e6923'},29755.0,{'$date': 1514851200000},MB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '625ee0f0a8ca9d5f6a1ea542'},29755.0,{'$date': 1514851200000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '625ee132a8ca9d5f6a20113e'},29755.0,{'$date': 1514851200000},MC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
#smartDocuments = SMARTAtt.objects()
smartDocumentsnone = SMARTAtt.objects(disk_id="12463").filter()

jSon_smartDocumentsnoneJson = json.loads(smartDocumentsnone.to_json())
df_smartDocumentsnoneJson = pd.DataFrame.from_dict(jSon_smartDocumentsnoneJson) 


#dicDateToString = json.dumps(list(df_smartDocumentsJson['timestamp']))
#dicStringToJson = json.loads(dicDateToString)
#dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
#df_smartDocumentsJson['timestamp'] = dicJsonToDf['$date']

df_smartDocumentsnoneJson.head()

,_id,disk_id,timestamp,model_x,r_sectors,u_errors,p_failedA,p_failedB,e_failedA,e_failedB,n_b_written,n_b_read
0,{'$oid': '625ee079a8ca9d5f6a1c3d99'},12463.0,{'$date': 1514764800000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '625ee0eca8ca9d5f6a1e8ffa'},12463.0,{'$date': 1514851200000},MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '625ee102a8ca9d5f6a1f090e'},12463.0,{'$date': 1514851200000},MC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '625ee18fa8ca9d5f6a221d6c'},12463.0,{'$date': 1514851200000},MB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '625ee1f3a8ca9d5f6a244271'},12463.0,{'$date': 1514937600000},MB2,100.0,100.0,NaN,100.0,NaN,100.0,99.0,99.0


In [100]:
#Mergin all smart attributes dataframes
df_smartDocumentsTotal = pd.concat([df_smartDocumentsJson, df_smartDocumentsDBJson, df_smartDocumentsNASJson, df_smartDocumentsRMJson, df_smartDocumentsSSJson, df_smartDocumentsWPSJson, df_smartDocumentsWSJson, df_smartDocumentsWSMJson, df_smartDocumentsnoneJson])

In [105]:
# Choosing the columns of interest
df_smartDocumentsTotalFiltered = df_smartDocumentsTotal.loc[:,['disk_id', 'timestamp' , 'r_sectors', 'u_errors','n_b_written','n_b_read']]

dicDateToString = json.dumps(list(df_smartDocumentsTotalFiltered['timestamp']))
dicStringToJson = json.loads(dicDateToString)
dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
df_smartDocumentsTotalFiltered['timestamp'] = dicJsonToDf['$date']

#Subctracting by the initial time of the experiment and turning into hours
df_smartDocumentsTotalFiltered['timestamp'] = df_smartDocumentsTotalFiltered.timestamp.sub(1514764800000)
df_smartDocumentsTotalFiltered['timestamp'] = df_smartDocumentsTotalFiltered.timestamp.div(1000)
df_smartDocumentsTotalFiltered['timestamp'] = df_smartDocumentsTotalFiltered.timestamp.div(60)
#df_smartDocumentsTotalFiltered['timestamp'] = df_smartDocumentsTotalFiltered.timestamp.div(60)
#df_smartDocumentsTotalFiltered['timestamp'] = df_smartDocumentsTotalFiltered.timestamp.div(24)
#df_smartDocumentsTotalFiltered['timestamp'] = df_smartDocumentsTotalFiltered.timestamp.div(30)

# Relating to app
appAtt = pd.DataFrame({
'disk_id':[19015, 95341, 18144 , 99516, 199807, 30010, 17304, 29755, 12463],  
'app': ['DAE', 'DB', 'NAS', 'RM', 'SS', 'WPS', 'WS', 'WSM', 'none']})

df_smartDocumentsTotalFiltered = pd.merge(df_smartDocumentsTotalFiltered, appAtt, how='left', on=['disk_id'])

df_smartDocumentsTotalFiltered.to_csv('smartattributesanalysisMinutesApp.csv')
#df_smartDocumentsTotalFiltered.tail(10)

In [106]:
df_smartDocumentsTotalFiltered.head()

,disk_id,timestamp,r_sectors,u_errors,n_b_written,n_b_read,app
0,19015.0,0.0,NaN,NaN,NaN,NaN,DAE
1,19015.0,0.0,NaN,NaN,NaN,NaN,DAE
2,19015.0,1440.0,NaN,NaN,NaN,NaN,DAE
3,19015.0,1440.0,NaN,NaN,NaN,NaN,DAE
4,19015.0,1440.0,NaN,NaN,NaN,NaN,DAE


In [318]:
df_failuresLocationDocumentsJson.to_csv('failuresLocation.csv', index = None, header=True)